In [1]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint

/home/sahand/anaconda3/envs/scopus201/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/home/sahand/anaconda3/envs/scopus201/lib/python3.7/site-packages/thinc/neural/train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from .optimizers import Adam, linear_decay
/home/sahand/anaconda3/envs/scopus201/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
/home/sahand/anaconda3/envs/scopus201/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections

## Note this notebook runs an HDP model to find the best number of topics. It is not strictly part of the flow and can be considered optional. It also uses a smaller sample of data to run faster.

#### --- Section not needed 

In [294]:
with open('rev_2016_df.pkl', 'rb') as f:
    rev_train = pickle.load(f)
with open('rev_2017.pkl', 'rb') as f:
    rev_test = pickle.load(f)

In [296]:
rev_train = rev_train.sample(frac=.3, random_state=1).reset_index(drop=True)
rev_test = rev_test.sample(frac=.3, random_state=1).reset_index(drop=True)

In [327]:
len(rev_train)

155924

### Read data and subsample data for faster operations


In [2]:
rev_train = pd.read_csv('/mnt/6016589416586D52/Users/z5204044/Documents/Dataset/Scopus_WoS_Merged/topic_modeling/abstracts - lemmatized - stemmed - 12 Aug 2019.csv',names=['text','orig_index'])[:200000]
rev_test = pd.read_csv('/mnt/6016589416586D52/Users/z5204044/Documents/Dataset/Scopus_WoS_Merged/topic_modeling/abstracts - lemmatized - stemmed - 12 Aug 2019.csv',names=['text','orig_index'])[200000:]

In [3]:
rev_train = rev_train.sample(100000)
rev_test = rev_test.sample(50000)

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [5]:
stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place'])

#### Remove new lines --- Not needed

In [7]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

In [ ]:
rev_train['text'] = strip_newline(rev_train.text)
rev_test['text'] = strip_newline(rev_test.text)

In [8]:
rev_train.text[21:22].values

array(['intellig control system use recent develop field artifici intellig draw signific interest overview given statu applic intellig control robot futur perspect basic concept underli intellig control highlight'],
      dtype=object)

#### Tokenize and remove punctuation

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [7]:
words_tr = list(sent_to_words(rev_train.text))
words_te = list(sent_to_words(rev_test.text))

In [8]:
words_tr[21][:10]

['right',
 'reserv',
 'gene',
 'express',
 'profil',
 'base',
 'high',
 'throughput',
 'technolog',
 'contribut']

In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [10]:
words_tr = remove_stopwords(words_tr)

In [11]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    trigram = gensim.models.Phrases(bigram[words], min_count = tri_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod

In [12]:
bigram_tr, trigram_tr = bigrams(words_tr)

/home/sahand/anaconda3/envs/scopus201/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


#### Check some items

In [13]:
print(trigram_tr[bigram_tr[words_tr[16345]]][:200])

['consider', 'product', 'design', 'develop', 'manufactur_industri', 'remark', 'increas', 'recent_year', 'improv', 'intellig', 'product', 'design', 'busi', 'becom', 'press_need', 'enterpris', 'concern', 'geometr', 'inform', 'like', 'draw', 'import', 'thing', 'consid', 'design', 'treat', 'exist', 'design', 'knowledg', 'mani', 'gener', 'purpos', 'cad', 'system', 'good', 'support', 'work', 'like', 'draw', 'pay', 'le_attent', 'featur', 'design', 'activ', 'effect', 'process', 'support', 'design', 'knowledg', 'thu', 'consider', 'growth', 'interest', 'knowledg', 'base', 'design', 'support', 'system', 'base', 'object_orient', 'methodolog', 'attribut', 'orient', 'model', 'paradigm', 'propos', 'assembl', 'structur', 'design', 'object', 'model', 'design', 'space', 'design', 'problem', 'represent', 'paper', 'focu', 'discuss', 'machin', 'element', 'model', 'knowledg_represent', 'construct', 'knowledg', 'base', 'design', 'support', 'system', 'verifi', 'effect', 'propos', 'method', 'applic', 'exampl']

#### Remove stopwords and lemmatize

In [14]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [15]:
def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#### Run test through trained model - will later run test data through trained model here

In [16]:
trigrams_tr = [trigram_tr[bigram_tr[review]] for review in words_tr]

In [17]:
lemma_lg = lemmatization(trigrams_tr)

In [18]:
with open('../results/lemma_lg.pkl', 'wb') as f:
    pickle.dump(lemma_lg, f)

#### Note difference to un-lemmatized un-stop-worded above

In [19]:
lemma_lg[8811][:20]

['artifici_intellig_ai',
 'system',
 'wide_accept',
 'technolog',
 'offer',
 'altern',
 'way',
 'tackl',
 'complex',
 'dynam',
 'problem',
 'urban',
 'studi',
 'goal',
 'articl_review',
 'current',
 'literatur',
 'field',
 'plan',
 'ai']

## LDA

#### Dictionary and Corpus creation for LDA

In [20]:
id2word_lg = gensim.corpora.Dictionary(lemma_lg)
id2word_lg.filter_extremes(no_below=10, no_above=0.35)
id2word_lg.compactify()
id2word_lg.save('../results/train_dict_lg')
corpus_lg = [id2word_lg.doc2bow(text) for text in lemma_lg]

/home/sahand/anaconda3/envs/scopus201/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [21]:
with open('../results/corpus_lg.pkl', 'wb') as f:
    pickle.dump(corpus_lg, f)

In [22]:
corpus_lg[21][:2]

[(6, 1), (9, 2)]

In [23]:
[(id2word_lg[id], freq) for id, freq in corpus_lg[21]]

[('applic', 1),
 ('chain', 2),
 ('complex', 1),
 ('contribut', 1),
 ('datum', 1),
 ('differ', 1),
 ('method', 1),
 ('propos', 1),
 ('techniqu', 2),
 ('approach', 1),
 ('consequ', 2),
 ('correl', 1),
 ('exist', 1),
 ('evalu', 2),
 ('experi_conduct', 1),
 ('maxim', 1),
 ('minim', 1),
 ('perform', 1),
 ('similar', 1),
 ('deriv', 1),
 ('good', 1),
 ('predict', 1),
 ('procedur', 1),
 ('set', 1),
 ('valid', 3),
 ('improv', 1),
 ('swarm_intellig', 1),
 ('test', 1),
 ('biolog', 1),
 ('provid', 1),
 ('statist', 1),
 ('comput', 1),
 ('line', 1),
 ('type', 1),
 ('identifi', 2),
 ('power', 1),
 ('simul', 1),
 ('high', 1),
 ('multipl', 1),
 ('technolog', 1),
 ('acquisit', 1),
 ('advantag', 1),
 ('assum', 1),
 ('bead', 2),
 ('cancer', 1),
 ('candid_gene', 2),
 ('cell', 1),
 ('classif', 2),
 ('clinic', 1),
 ('context', 1),
 ('convey', 1),
 ('devi', 1),
 ('diagnost', 1),
 ('differenti_express', 1),
 ('dimens_reduct', 1),
 ('discrimin_power', 1),
 ('disea', 1),
 ('experiment_evid', 1),
 ('express', 1),

## HDP Model - auto-dinals the best number of topics

In [32]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_lg, id2word_lg, chunksize=10000, T=100)

/home/sahand/anaconda3/envs/scopus201/lib/python3.7/site-packages/gensim/models/hdpmodel.py:222: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [34]:
len(hdp.print_topics(num_topics=-1))

100

In [ ]:
hdp.print_topics(num_topics=20)

In [35]:
def topic_prob_extractor(gensim_hdp):
    shown_topics = gensim_hdp.show_topics(num_topics=-1, formatted=False)
    topics_nos = [x[0] for x in shown_topics ]
    weights = [ sum([item[1] for item in shown_topics[topicN][1]]) for topicN in topics_nos ]

    return pd.DataFrame({'topic_id' : topics_nos, 'weight' : weights})

In [36]:
topic_prob_extractor(hdp)

,topic_id,weight
0,0,0.103638
1,1,0.038081
2,2,0.029518
3,3,0.020865
4,4,0.022594
5,5,0.021473
6,6,0.019395
7,7,0.016351
8,8,0.011433
9,9,0.011106


In [39]:
import sys
sys.modules[__name__].__dict__.clear()